# textCNN

In [ ]:
# coding: utf-8

import tensorflow as tf


class TCNNConfig(object):
    """CNN配置参数"""

    embedding_dim = 64  # 词向量维度
    seq_length = 600  # 序列长度
    num_classes = 10  # 类别数
    num_filters = 256  # 卷积核数目
    kernel_size = 5  # 卷积核尺寸
    vocab_size = 5000  # 词汇表达小

    hidden_dim = 128  # 全连接层神经元

    dropout_keep_prob = 0.5  # dropout保留比例
    learning_rate = 1e-3  # 学习率

    batch_size = 64  # 每批训练大小
    num_epochs = 10  # 总迭代轮次

    print_per_batch = 100  # 每多少轮输出一次结果
    save_per_batch = 10  # 每多少轮存入tensorboard


class TextCNN(object):
    """文本分类，CNN模型"""

    def __init__(self, config):
        self.config = config

        # 三个待输入的数据
        self.input_x = tf.placeholder(tf.int32, [None, self.config.seq_length], name='input_x')  # [bs,seq_len] each elem is id
        self.input_y = tf.placeholder(tf.float32, [None, self.config.num_classes], name='input_y')  # [bs, num_classes] one-hot
        self.keep_prob = tf.placeholder(tf.float32, name='keep_prob')

        self.cnn()

    def cnn(self):
        """CNN模型"""
        # 词向量映射
        with tf.device('/cpu:0'):
            embedding = tf.get_variable('embedding', [self.config.vocab_size, self.config.embedding_dim])
            embedding_inputs = tf.nn.embedding_lookup(embedding, self.input_x)  # [bs,seq_len,emb_dim]

        with tf.name_scope("cnn"):
            # CNN layer  [bs, seq_len, emb_dim] -> [bs, len(seq)-kernel_size+1, num_filters]
            conv = tf.layers.conv1d(embedding_inputs, self.config.num_filters, self.config.kernel_size, name='conv')
            # global max pooling layer [bs, len(seq)-kernel_size+1, num_filters] -> [bs, num_filters]
            gmp = tf.reduce_max(conv, reduction_indices=[1], name='gmp')

        with tf.name_scope("score"):
            # 全连接层，后面接dropout以及relu激活[bs, num_filters] -> [bs, hidden_dim]
            fc = tf.layers.dense(gmp, self.config.hidden_dim, name='fc1')
            fc = tf.contrib.layers.dropout(fc, self.keep_prob)
            fc = tf.nn.relu(fc)

            # 分类器[bs, hidden_dim] -> [bs, num_classes]
            self.logits = tf.layers.dense(fc, self.config.num_classes, name='fc2')
            self.y_pred_cls = tf.argmax(tf.nn.softmax(self.logits), 1)  # 预测类别

        with tf.name_scope("optimize"):
            # 损失函数，交叉熵
            cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.input_y)
            self.loss = tf.reduce_mean(cross_entropy)
            # 优化器
            self.optim = tf.train.AdamOptimizer(learning_rate=self.config.learning_rate).minimize(self.loss)

        with tf.name_scope("accuracy"):
            # 准确率
            correct_pred = tf.equal(tf.argmax(self.input_y, 1), self.y_pred_cls)
            self.acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

参见上面的代码。

每个句子长度限定为`seq_len`，长的截断，短的补零。

每次有`batch_size`个句子参与训练，所以每次训练数据是`(batch_size, seq_len)`

`embedding`词表长`vocab_size`，宽`emb_dim`，可以理解为我、是、你这种中文字每个字编号/hash为一个整数，一共有`vocab_size`个中文字有向量表示，每个向量长`emb_dim`。

通过`embedding_lookup`将每个整数id表示的中文字/word转化为对应的向量重新构成`(batch_dize, seq_len, emb_dim)`的数据。类比图像数据则有，`seq_len`为数据尺寸对应`width, height`,`emb_dim`对应`channel`及之后的`feature_map`

一维卷积kernel只有一个维度，在经过`num_filters`个`kernel_size`的kernel处理后, 数据尺寸从`seq_len`变为`seq_len-kernel_size+1`同图像尺寸计算方式，`feature_map`从`emb_dim`变为`num_filters`，每种filter对应一种特征。

在该段代码中，文本句子经过一层`conv1d`后进行`reduce_max`最大池化，使得shape从`(batch_size, seq_len-kernel_size+1, num_filters)`变为`(batch_size, num_filters)`,然后进行`dense`等操作，其实还可以继续重复上面的`conv1d`构成其他模型。